## Statistical Analysis of AES-128
#### Meenu Ahluwalia, Chris Boomhower, Steven Millett
This Jupyter Notebook contains the code used to implement our AES-128 bias analysis for MSDS7349

In [ ]:
import base64
import hashlib
import pandas as pd

# To install pycrypto module, run following from command line in PuTTy:  python3 /usr/local/es7/bin/pip3 install --user pycrypto
from Crypto import Random
from Crypto.Cipher import AES

In [ ]:
# Following code chunk obtained from https://stackoverflow.com/questions/12524994/encrypt-decrypt-using-pycrypto-aes-256/12525165#12525165

BS = 16
pad = lambda s: s + (BS - len(s) % BS) * chr(BS - len(s) % BS)
unpad = lambda s : s[0:-s[-1]]


class AESCipher:

    def __init__( self, key ):
        self.key = hashlib.sha256(key.encode('utf-8')).digest()

    def encrypt( self, raw ):
        raw = pad(raw)
        iv = Random.new().read( AES.block_size )
        cipher = AES.new( self.key, AES.MODE_CBC, iv )
        return base64.b64encode( iv + cipher.encrypt( raw ) )

    def decrypt( self, enc ):
        enc = base64.b64decode(enc)
        iv = enc[:16]
        cipher = AES.new(self.key, AES.MODE_CBC, iv )
        return unpad(cipher.decrypt( enc[16:] ))

Quick look at encryption and decryption using the above class

In [ ]:
cipher = AESCipher('mysecretpassword') #16-byte password
encrypted = cipher.encrypt('Super secret message')
decrypted = cipher.decrypt(encrypted)
print(encrypted)
print(decrypted)

Quick look at how long it takes to generate 1 million cipher blocks.

In [ ]:
%%time
cipher = AESCipher('mysecretpassword') #16-byte password

AES_list = []
for x, i in enumerate(range(0,1000000)):
    AES_list.append(cipher.encrypt('Super secret message %d' %i))

dec_list = []
for crypto in AES_list:
    dec_list.append(cipher.decrypt(crypto))

Seems to be working well....

In [ ]:
%%time
for crypto in AES_list[:10]:
    print(crypto)
for msg in dec_list[:10]:
    print(msg)

In [ ]:
len(AES_list)